In [1]:
# imports
import os
import pandas as pd


In [2]:
import pandas as pd
import os

def preprocess_csv(file_path):
    """
    Preprocesses a single CSV file by skipping initial empty rows,
    combining two header rows, and removing empty columns.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        pd.DataFrame: The preprocessed DataFrame, or None if an error occurs.
    """
    try:
        # Read the file, skipping the initial empty rows.
        # It's observed that there are 4 empty rows, so we start reading from row 4 (0-indexed).
        # We read the next two rows (index 4 and 5) as potential header rows.
        df_header_part1 = pd.read_csv(file_path, header=None, skiprows=3, nrows=1)
        df_header_part2 = pd.read_csv(file_path, header=None, skiprows=4, nrows=1)

        # Read the actual data, skipping the initial empty rows and the two header rows
        df_data = pd.read_csv(file_path, header=None, skiprows=5)

        # Combine the two header parts
        # For columns that are not years (e.g., 'Varianten der Bevölkerungsvorausberechnung', 'Geschlecht', 'Altersjahre'),
        # the header is simply the value from df_header_part1.
        # For year columns, combine both parts: e.g., '2022 - 31.12.2022'

        # Initialize the new header list
        new_header = []

        # Iterate through the columns and combine headers
        for col_idx in range(len(df_header_part1.columns)):
            # Check if both parts of the header exist for this column
            header_part1_val = str(df_header_part1.iloc[0, col_idx]).strip() if col_idx < len(df_header_part1.columns) else ''
            header_part2_val = str(df_header_part2.iloc[0, col_idx]).strip() if col_idx < len(df_header_part2.columns) else ''

            if header_part1_val and header_part2_val and header_part1_val != 'nan' and header_part2_val != 'nan':
                # If the second part looks like a date, combine them
                if any(char.isdigit() for char in header_part2_val) and '.' in header_part2_val:
                    new_header.append(f"{header_part1_val} - {header_part2_val}")
                else:
                    new_header.append(header_part1_val) # Use the first part if second is not a date
            elif header_part1_val and header_part1_val != 'nan':
                new_header.append(header_part1_val)
            elif header_part2_val and header_part2_val != 'nan':
                new_header.append(header_part2_val)
            else:
                new_header.append(f"Unnamed_Col_{col_idx}") # Fallback for completely empty header parts


        # Assign the new header to the data DataFrame
        df_data.columns = new_header[:len(df_data.columns)] # Ensure header length matches data columns

        # Remove columns that are entirely empty or contain only 'e'
        # First, drop columns where all values are NaN
        df_data.dropna(axis=1, how='all', inplace=True)

        # Then, drop columns where all values are 'e' (if they exist after the initial dropna)
        # Convert all values to string to handle 'e' consistently
        df_data = df_data.loc[:, ~df_data.astype(str).apply(lambda x: (x == 'e').all())]
        
        # Also remove columns where the header might be 'Unnamed_Col_X' and data is empty
        # This covers cases where 'e' was removed and no proper header was found
        cols_to_drop_by_name = [col for col in df_data.columns if 'Unnamed_Col_' in str(col) and df_data[col].isnull().all()]
        df_data.drop(columns=cols_to_drop_by_name, inplace=True)


        print(f"Successfully preprocessed {file_path}")
        return df_data

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def process_multiple_csv_files(input_directory, output_directory):
    """
    Processes all CSV files in a given input directory and saves the
    preprocessed files to an output directory.

    Args:
        input_directory (str): The path to the directory containing input CSV files.
        output_directory (str): The path to the directory where preprocessed files will be saved.
    """
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Created output directory: {output_directory}")

    for filename in os.listdir(input_directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(input_directory, filename)
            print(f"Processing file: {file_path}")
            preprocessed_df = preprocess_csv(file_path)

            if preprocessed_df is not None:
                output_file_path = os.path.join(output_directory, f"cleaned_{filename}")
                preprocessed_df.to_csv(output_file_path, index=False)
                print(f"Saved cleaned data to: {output_file_path}")

# --- How to use the code ---
# 1. Replace 'your_input_csv_directory' with the path to the folder containing your CSV files.
# 2. Replace 'your_output_csv_directory' with the path to the folder where you want to save the cleaned files.
#    If the output directory doesn't exist, the code will create it.

# Example usage:
input_dir = r"C:\Users\user\Desktop\A_AI_DDSP\data\processed\raw" # e.g., 'data/raw_csvs'
output_dir = r"C:\Users\user\Desktop\A_AI_DDSP\data\processed\Processed" # e.g., 'data/processed_csvs'

# # For demonstration, let's assume the user has the 'Bevölkerung forecasts_VAR 02.csv' in the current directory
# # and wants to save the output in a folder named 'output_csvs' in the same directory.
# input_dir = '.' # Current directory
# output_dir = 'output_csvs'

process_multiple_csv_files(input_dir, output_dir)

Processing file: C:\Users\user\Desktop\A_AI_DDSP\data\processed\raw\Bevölkerung forecasts_ALL VAR.csv
Successfully preprocessed C:\Users\user\Desktop\A_AI_DDSP\data\processed\raw\Bevölkerung forecasts_ALL VAR.csv
Saved cleaned data to: C:\Users\user\Desktop\A_AI_DDSP\data\processed\Processed\cleaned_Bevölkerung forecasts_ALL VAR.csv
Processing file: C:\Users\user\Desktop\A_AI_DDSP\data\processed\raw\Bevölkerung forecasts_VAR 02.csv
Successfully preprocessed C:\Users\user\Desktop\A_AI_DDSP\data\processed\raw\Bevölkerung forecasts_VAR 02.csv
Saved cleaned data to: C:\Users\user\Desktop\A_AI_DDSP\data\processed\Processed\cleaned_Bevölkerung forecasts_VAR 02.csv
Processing file: C:\Users\user\Desktop\A_AI_DDSP\data\processed\raw\Bevölkerung historics_per BL.csv
Successfully preprocessed C:\Users\user\Desktop\A_AI_DDSP\data\processed\raw\Bevölkerung historics_per BL.csv
Saved cleaned data to: C:\Users\user\Desktop\A_AI_DDSP\data\processed\Processed\cleaned_Bevölkerung historics_per BL.csv
P

24


In [6]:
import mysql.connector
from mysql.connector import Error
import csv
import os

# --- MySQL Database Configuration ---
DB_CONFIG = {
    'host': 'localhost',
    'user': 'root', # User with CREATE DATABASE privilege (e.g., 'root')
    'password': 'Pavani@3534',
    'database': 'hospital_data' # Name of the database to create/use
}

# --- CSV File Paths ---
# NOTE: Updated with the exact paths provided in the error output
CSV_FILE_PATHS = {
    'population': [
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_Bevölkerung forecasts_ALL VAR.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_Bevölkerung forecasts_VAR 02.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_Bevölkerung historics_per BL.csv'
    ],
    'bed_details': [
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_DSB_RHV_2023.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_DSB_RHV_2022.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_DSB_RHV_2021.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_DSB_KHV_2023.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_DSB_KHV_2022.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_DSB_KHV_2021.csv'
    ],
    'comprehensive': [
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_KHV_2021.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_KHV_2022.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_KHV_2023.csv'
    ],
    'summarized': [
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_RHV_2023.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_RHV_2022.csv',
        'C:\\Users\\user\\Desktop\\A_AI_DDSP\\data\\processed\\Processed\\cleaned_RHV_2021.csv'
    ]
}

# --- SQL CREATE TABLE Statements (Updated to FLOAT for population data) ---
SQL_CREATE_TABLES = """
-- Table for File 1: Population Projection Data
-- This table stores population counts based on different projection variants, gender, age, and specific dates.
CREATE TABLE IF NOT EXISTS PopulationProjections (
    id INT PRIMARY KEY AUTO_INCREMENT, -- Unique identifier for each row
    ProjectionVariant VARCHAR(255),    -- Description of the population projection variant
    Gender VARCHAR(50),                -- Gender (e.g., 'männlich', 'weiblich', 'Insgesamt')
    AgeGroup VARCHAR(50),              -- Age or age group (e.g., 'unter 1 Jahr', '1-Jährige', '90+', 'Insgesamt')
    Population_2022_12_31 FLOAT,         -- Population count for 2022-12-31
    Population_2023_12_31 FLOAT,         -- Population count for 2023-12-31
    Population_2024_12_31 FLOAT,         -- Population count for 2024-12-31
    Population_2025_12_31 FLOAT,         -- Population count for 2025-12-31
    Population_2026_12_31 FLOAT,         -- Population count for 2026-12-31
    Population_2027_12_31 FLOAT,         -- Population count for 2027-12-31
    Population_2028_12_31 FLOAT,         -- Population count for 2028-12-31
    Population_2029_12_31 FLOAT,         -- Population count for 2029-12-31
    Population_2030_12_31 FLOAT,         -- Population count for 2030-12-31
    Population_2031_12_31 FLOAT,         -- Population count for 2031-12-31
    Population_2032_12_31 FLOAT,         -- Population count for 2032-12-31
    Population_2033_12_31 FLOAT,         -- Population count for 2033-12-31
    Population_2034_12_31 FLOAT,         -- Population count for 2034-12-31
    Population_2035_12_31 FLOAT,         -- Population count for 2035-12-31
    Population_2036_12_31 FLOAT,         -- Population count for 2036-12-31
    Population_2037_12_31 FLOAT,         -- Population count for 2037-12-31
    Population_2038_12_31 FLOAT,         -- Population count for 2038-12-31
    Population_2039_12_31 FLOAT,         -- Population count for 2039-12-31
    Population_2040_12_31 FLOAT,         -- Population count for 2040-12-31
    Population_2041_12_31 FLOAT,         -- Population count for 2041-12-31
    Population_2042_12_31 FLOAT,         -- Population count for 2042-12-31
    Population_2043_12_31 FLOAT,         -- Population count for 2043-12-31
    Population_2044_12_31 FLOAT,         -- Population count for 2044-12-31
    Population_2045_12_31 FLOAT,         -- Population count for 2045-12-31
    Population_2046_12_31 FLOAT,         -- Population count for 2046-12-31
    Population_2047_12_31 FLOAT,         -- Population count for 2047-12-31
    Population_2048_12_31 FLOAT,         -- Population count for 2048-12-31
    Population_2049_12_31 FLOAT,         -- Population count for 2049-12-31
    Population_2050_12_31 FLOAT,         -- Population count for 2050-12-31
    Population_2051_12_31 FLOAT,         -- Population count for 2051-12-31
    Population_2052_12_31 FLOAT,         -- Population count for 2052-12-31
    Population_2053_12_31 FLOAT,         -- Population count for 2053-12-31
    Population_2054_12_31 FLOAT,         -- Population count for 2054-12-31
    Population_2055_12_31 FLOAT,         -- Population count for 2055-12-31
    Population_2056_12_31 FLOAT,         -- Population count for 2056-12-31
    Population_2057_12_31 FLOAT,         -- Population count for 2057-12-31
    Population_2058_12_31 FLOAT,         -- Population count for 2058-12-31
    Population_2059_12_31 FLOAT,         -- Population count for 2059-12-31
    Population_2060_12_31 FLOAT,         -- Population count for 2060-12-31
    Population_2061_12_31 FLOAT,         -- Population count for 2061-12-31
    Population_2062_12_31 FLOAT,         -- Population count for 2062-12-31
    Population_2063_12_31 FLOAT,         -- Population count for 2063-12-31
    Population_2064_12_31 FLOAT,         -- Population count for 2064-12-31
    Population_2065_12_31 FLOAT,         -- Population count for 2065-12-31
    Population_2066_12_31 FLOAT,         -- Population count for 2066-12-31
    Population_2067_12_31 FLOAT,         -- Population count for 2067-12-31
    Population_2068_12_31 FLOAT,         -- Population count for 2068-12-31
    Population_2069_12_31 FLOAT,         -- Population count for 2069-12-31
    Population_2070_12_31 FLOAT          -- Population count for 2070-12-31
);

-- Table for File 2: Hospital Bed Details
-- This table serves as a lookup for detailed hospital department codes and their descriptions.
CREATE TABLE IF NOT EXISTS HospitalBedDetails (
    id INT PRIMARY KEY AUTO_INCREMENT, -- Unique identifier for each row
    FieldName VARCHAR(255) UNIQUE,     -- The field name or code (e.g., 'Kreis', '0100', 'INSG')
    Content TEXT                       -- The description or content associated with the field name
);

-- Table for File 3: Comprehensive Hospital Facilities Data
-- This table stores comprehensive information about hospital facilities, including location, contact,
-- ownership, type, emergency care provisions, and detailed bed counts by department.
CREATE TABLE IF NOT EXISTS HospitalFacilitiesComprehensive (
    id INT PRIMARY KEY AUTO_INCREMENT, -- Unique identifier for each hospital entry
    Land VARCHAR(100),                 -- State or federal state in Germany
    Kreis VARCHAR(100),                -- District/County (with first digit potentially indicating administrative region)
    Gemeinde VARCHAR(100),             -- Municipality
    HospitalName VARCHAR(255),         -- Name of the hospital
    SiteName VARCHAR(255),             -- Name of the hospital site/location
    SiteStreet VARCHAR(255),           -- Street name of the hospital site
    SiteHouseNumber VARCHAR(50),       -- House number of the hospital site
    SitePostalCode VARCHAR(20),        -- Postal code of the hospital site
    SiteCity VARCHAR(100),             -- City/Town of the hospital site
    PhoneNumber VARCHAR(50),           -- Full telephone number (area code and number)
    EmailAddress VARCHAR(255),         -- Email address of the hospital
    InternetAddress VARCHAR(255),      -- Website address of the hospital
    OwnerType INT,                     -- Type of ownership (1=public, 2=non-profit, 3=private)
    OwnerName VARCHAR(255),            -- Name of the owner/operator
    InstitutionType INT,               -- Type of institution (e.g., 1=university clinic, 2=planned hospital, etc.)
    GeneralEmergencyCare INT,          -- Level of general stationary emergency care (0=none, 1=basic, 2=extended, 3=comprehensive)
    -- Note: Assuming these are distinct columns for special emergency care modules.
    -- If they are meant to be a single column with multiple values, consider a separate linking table or JSON/TEXT field.
    SpecialEmergencyCare_Modul1 VARCHAR(255), -- Special stationary emergency care module 1
    SpecialEmergencyCare_Modul2 VARCHAR(255), -- Special stationary emergency care module 2
    SpecialEmergencyCare_Modul3 VARCHAR(255), -- Special stationary emergency care module 3
    SpecialEmergencyCare_Modul4 VARCHAR(255), -- Special stationary emergency care module 4
    SpecialEmergencyCare_Modul5 VARCHAR(255), -- Special stationary emergency care module 5
    TotalBeds INT,                     -- Total number of established beds (INSG)

    -- Detailed Bed Counts by Department (based on File 2's descriptions)
    Beds_100_118 INT,                  -- Internal Medicine
    Beds_102 INT,                      -- Internal Medicine/Geriatrics
    Beds_103 INT,                      -- Internal Medicine/Cardiology
    Beds_104 INT,                      -- Internal Medicine/Nephrology
    Beds_105 INT,                      -- Internal Medicine/Hematology and Internal Oncology
    Beds_106 INT,                      -- Internal Medicine/Endocrinology
    Beds_107 INT,                      -- Internal Medicine/Gastroenterology
    Beds_108 INT,                      -- Internal Medicine/Pneumology
    Beds_109 INT,                      -- Internal Medicine/Rheumatology
    Beds_114 INT,                      -- Internal Medicine/Pulmonary and Bronchial Medicine
    Beds_152 INT,                      -- Internal Medicine/Infectious Diseases
    Beds_153 INT,                      -- Internal Medicine/Diabetes
    Beds_154 INT,                      -- Internal Medicine/Naturopathy
    Beds_156 INT,                      -- Internal Medicine/Stroke units
    Beds_200 INT,                      -- Geriatrics
    Beds_300 INT,                      -- Cardiology
    Beds_400 INT,                      -- Nephrology
    Beds_500 INT,                      -- Hematology and Internal Oncology
    Beds_510 INT,                      -- Hematology and Internal Oncology/Pediatrics
    Beds_533 INT,                      -- Hematology and Internal Oncology/Radiotherapy
    Beds_600 INT,                      -- Endocrinology
    Beds_700 INT,                      -- Gastroenterology
    Beds_800 INT,                      -- Pneumology
    Beds_900 INT,                      -- Rheumatology
    Beds_1000_34 INT,                  -- Pediatrics
    Beds_1004 INT,                     -- Pediatrics/Nephrology
    Beds_1005 INT,                     -- Pediatrics/Hematology and Internal Oncology
    Beds_1007 INT,                     -- Pediatrics/Gastroenterology
    Beds_1009 INT,                     -- Pediatrics/Rheumatology
    Beds_1011 INT,                     -- Pediatrics/Pediatric Cardiology
    Beds_1012 INT,                     -- Pediatrics/Neonatology
    Beds_1014 INT,                     -- Pediatrics/Pulmonary and Bronchial Medicine
    Beds_1028 INT,                     -- Pediatrics/Pediatric Neurology
    Beds_1051 INT,                     -- Long-term care for children
    Beds_1100 INT,                     -- Pediatric Cardiology
    Beds_1136 INT,                     -- Pediatric Cardiology/Intensive Care Medicine
    Beds_1200 INT,                     -- Neonatology
    Beds_1300 INT,                     -- Pediatric Surgery
    Beds_1400 INT,                     -- Pulmonary and Bronchial Medicine
    Beds_1500 INT,                     -- General Surgery
    Beds_1513 INT,                     -- General Surgery/Pediatric Surgery
    Beds_1516 INT,                     -- General Surgery/Trauma Surgery
    Beds_1518 INT,                     -- General Surgery/Vascular Surgery
    Beds_1519 INT,                     -- General Surgery/Plastic Surgery
    Beds_1520 INT,                     -- General Surgery/Thoracic Surgery
    Beds_1523 INT,                     -- Surgery/Orthopedics
    Beds_1536 INT,                     -- General Surgery/Intensive Care Medicine
    Beds_1550 INT,                     -- General Surgery/Abdominal and Vascular Surgery
    Beds_1551 INT,                     -- General Surgery/Hand Surgery
    Beds_1600_59 INT,                  -- Trauma Surgery
    Beds_1700_35 INT,                  -- Neurosurgery
    Beds_1800_28 INT,                  -- Vascular Surgery
    Beds_1900 INT,                     -- Plastic Surgery
    Beds_2000 INT,                     -- Thoracic Surgery
    Beds_2021 INT,                     -- Thoracic Surgery/Cardiac Surgery
    Beds_2036 INT,                     -- Thoracic Surgery/Intensive Care Medicine
    Beds_2050 INT,                     -- Thoracic Surgery/Cardiac Surgery Intensive Care Medicine
    Beds_2100 INT,                     -- Cardiac Surgery
    Beds_2118 INT,                     -- Cardiac Surgery/Vascular Surgery
    Beds_2120 INT,                     -- Cardiac Surgery/Thoracic Surgery
    Beds_2136 INT,                     -- Cardiac Surgery/Intensive Care Medicine
    Beds_2150 INT,                     -- Cardiac Surgery/Thoracic Surgery Intensive Care Medicine
    Beds_2200_39 INT,                  -- Urology
    Beds_2300 INT,                     -- Orthopedics
    Beds_2309 INT,                     -- Orthopedics/Rheumatology
    Beds_2315 INT,                     -- Orthopedics/Surgery
    Beds_2316 INT,                     -- Orthopedics and Trauma Surgery
    Beds_2400_41 INT,                  -- Gynecology and Obstetrics
    Beds_2402 INT,                     -- Gynecology/Geriatrics
    Beds_2405 INT,                     -- Gynecology/Hematology and Internal Oncology
    Beds_2406 INT,                     -- Gynecology/Endocrinology
    Beds_2425 INT,                     -- Gynecology
    Beds_2500 INT,                     -- Obstetrics
    Beds_2600 INT,                     -- Otorhinolaryngology (ENT)
    Beds_2700 INT,                     -- Ophthalmology
    Beds_2800_46 INT,                  -- Neurology
    Beds_2810 INT,                     -- Neurology/Pediatrics
    Beds_2851 INT,                     -- Neurology/Gerontology
    Beds_2852 INT,                     -- Neurology/Neurological Early Rehabilitation
    Beds_2856 INT,                     -- Neurology/Stroke Patients
    Beds_2900 INT,                     -- General Psychiatry
    Beds_2928 INT,                     -- General Psychiatry/Neurology
    Beds_2930 INT,                     -- General Psychiatry/Child and Adolescent Psychiatry
    Beds_2931 INT,                     -- General Psychiatry/Psychosomatics/Psychotherapy
    Beds_2950 INT,                     -- General Psychiatry/Addiction Treatment
    Beds_2951 INT,                     -- General Psychiatry/Gerontopsychiatry
    Beds_2952 INT,                     -- General Psychiatry/Forensic Treatment
    Beds_2953 INT,                     -- General Psychiatry/Addiction Treatment, Day Clinic
    Beds_2954 INT,                     -- General Psychiatry/Addiction Treatment, Night Clinic
    Beds_2955 INT,                     -- General Psychiatry/Gerontopsychiatry, Day Clinic
    Beds_2956 INT,                     -- General Psychiatry/Gerontopsychiatry, Night Clinic
    Beds_2960 INT,                     -- General Psychiatry/Day Clinic
    Beds_2961 INT,                     -- General Psychiatry/Night Clinic
    Beds_3000 INT,                     -- Child and Adolescent Psychiatry
    Beds_3060 INT,                     -- Child and Adolescent Psychiatry/Day Clinic
    Beds_3061 INT,                     -- Child and Adolescent Psychiatry/Night Clinic
    Beds_3100 INT,                     -- Psychosomatics/Psychotherapy
    Beds_3110 INT,                     -- Psychosomatics/Psychotherapy/Child and Adolescent Psychosomatics
    Beds_3160 INT,                     -- Psychosomatics/Psychotherapy/Day Clinic
    Beds_3161 INT,                     -- Psychosomatics/Psychotherapy/Night Clinic
    Beds_3200 INT,                     -- Nuclear Medicine
    Beds_3233 INT,                     -- Nuclear Medicine/Radiotherapy
    Beds_3300 INT,                     -- Radiotherapy
    Beds_3305 INT,                     -- Radiotherapy/Hematology and Internal Oncology
    Beds_3350 INT,                     -- Radiotherapy/Radiology
    Beds_3400 INT,                     -- Dermatology
    Beds_3460 INT,                     -- Dermatology/Day Clinic
    Beds_3500 INT,                     -- Dentistry and Oral and Maxillofacial Surgery
    Beds_3600 INT,                     -- Intensive Care Medicine
    Beds_3601 INT,                     -- Intensive Care Medicine/Internal Medicine
    Beds_3603 INT,                     -- Intensive Care Medicine/Cardiology
    Beds_3610 INT,                     -- Intensive Care Medicine/Pediatrics
    B3617 INT,                     -- Intensive Care Medicine/Neurosurgery
    Beds_3618 INT,                     -- Intensive Care Medicine/Surgery
    Beds_3621 INT,                     -- Intensive Care Medicine/Cardiac Surgery
    Beds_3622 INT,                     -- Intensive Care Medicine/Urology
    Beds_3624 INT,                     -- Intensive Care Medicine/Gynecology and Obstetrics
    Beds_3626 INT,                     -- Intensive Care Medicine/Otorhinolaryngology (ENT)
    Beds_3628 INT,                     -- Intensive Care Medicine/Neurology
    Beds_3650 INT,                     -- Operative Intensive Care Medicine/Surgery
    Beds_3651 INT,                     -- Intensive Care Medicine/Thoracic-Cardiac Surgery
    Beds_3652 INT,                     -- Intensive Care Medicine/Heart-Thoracic Surgery
    Beds_3700 INT,                     -- Other Special Department
    Beds_3750 INT,                     -- Angiology
    Beds_3751 INT,                     -- Radiology
    Beds_3752 INT,                     -- Palliative Medicine
    Beds_3753 INT,                     -- Pain Therapy
    Beds_3754 INT,                     -- Curative Therapy Department
    Beds_3755 INT,                     -- Spinal Surgery
    Beds_3756 INT,                     -- Addiction Medicine
    Beds_3757 INT,                     -- Visceral Surgery
    Beds_3758 INT                      -- Weaning Unit
);

-- Table for File 4: Summarized Hospital Facilities Data
-- This table provides a summarized view of hospital facilities with broader bed count categories.
CREATE TABLE IF NOT EXISTS HospitalFacilitiesSummarized (
    id INT PRIMARY KEY AUTO_INCREMENT, -- Unique identifier for each hospital entry
    Land VARCHAR(100),                 -- State or federal state in Germany
    Kreis VARCHAR(100),                -- District/County
    Gemeinde VARCHAR(100),             -- Municipality
    HospitalName VARCHAR(255),         -- Name of the hospital
    Street VARCHAR(255),               -- Street name
    HouseNumber VARCHAR(50),           -- House number
    PostalCode VARCHAR(20),            -- Postal code
    City VARCHAR(100),                 -- City/Town
    PhoneNumber VARCHAR(50),           -- Full telephone number (area code and number)
    EmailAddress VARCHAR(255),         -- Email address of the hospital
    InternetAddress VARCHAR(255),      -- Website address of the hospital
    OwnerType INT,                     -- Type of ownership (1=public, 2=non-profit, 3=private)
    OwnerName VARCHAR(255),            -- Name of the owner/operator
    InstitutionType INT,               -- Type of institution (e.g., 1=university clinic, 2=planned hospital, etc.)
    TotalBeds INT,                     -- Total number of established beds (INSG)

    -- Summarized Bed Counts by Major Department
    Beds_0 INT,                        -- Unknown/Unspecified beds (if this represents a general category)
    Beds_100 INT,                      -- Internal Medicine (main category)
    Beds_200 INT,                      -- Geriatrics (main category)
    Beds_300 INT,                      -- Cardiology (main category)
    Beds_400 INT,                      -- Nephrology (main category)
    Beds_500 INT,                      -- Hematology and Internal Oncology (main category)
    Beds_600 INT,                      -- Endocrinology (main category)
    Beds_700 INT,                      -- Gastroenterology (main category)
    Beds_800 INT,                      -- Pneumology (main category)
    Beds_900 INT,                      -- Rheumatology (main category)
    Beds_1000 INT,                     -- Pediatrics (main category)
    Beds_1400 INT,                     -- Pulmonary and Bronchial Medicine (main category)
    Beds_1500 INT,                     -- General Surgery (main category)
    Beds_1600 INT,                     -- Trauma Surgery (main category)
    Beds_1800 INT,                     -- Vascular Surgery (main category)
    Beds_2100 INT,                     -- Cardiac Surgery (main category)
    Beds_2200 INT,                     -- Urology (main category)
    Beds_2300 INT,                     -- Orthopedics (main category)
    Beds_2400 INT,                     -- Gynecology and Obstetrics (main category)
    Beds_2600 INT,                     -- Otorhinolaryngology (ENT) (main category)
    Beds_2700 INT,                     -- Ophthalmology (main category)
    Beds_2800 INT,                     -- Neurology (main category)
    Beds_2900 INT,                     -- General Psychiatry (main category)
    Beds_3000 INT,                     -- Child and Adolescent Psychiatry (main category)
    Beds_3100 INT,                     -- Psychosomatics/Psychotherapy (main category)
    Beds_3400 INT,                     -- Dermatology (main category)
    Beds_3700 INT,                     -- Other Special Department (main category)
    Beds_8200 INT,                     -- Unknown/Other department category
    Beds_8500_86 INT,                  -- Unknown/Other department category (range seems odd, consider renaming if possible)
    Beds_8600 INT,                     -- Unknown/Other department category
    Beds_8800 INT,                     -- Unknown/Other department category
    Beds_9000 INT                      -- Unknown/Other department category
);
"""

# --- Helper Functions for Connection and Table Creation ---

def create_database_if_not_exists(config):
    """
    Connects to MySQL (without specifying a database) and creates the specified database if it doesn't exist.
    """
    db_name = config['database']
    temp_config = {k: v for k, v in config.items() if k != 'database'} # Connect without specific database
    connection = None
    try:
        connection = mysql.connector.connect(**temp_config)
        cursor = connection.cursor()
        print(f"Attempting to create database '{db_name}' if it doesn't exist...")
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
        connection.commit()
        print(f"Database '{db_name}' ensured to exist.")
        return True
    except Error as e:
        print(f"Error creating database '{db_name}': {e}")
        return False
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()

def create_tables(config, sql_statements):
    """
    Connects to the specified database and executes the CREATE TABLE statements.
    Each table creation uses IF NOT EXISTS to prevent errors if the table already exists.
    """
    connection = None
    try:
        connection = mysql.connector.connect(**config) # Connect to the specific database
        cursor = connection.cursor()

        # Split SQL statements by semicolon and execute each
        statements = [s.strip() for s in sql_statements.split(';') if s.strip()]

        for statement in statements:
            try:
                # Add IF NOT EXISTS to each CREATE TABLE statement if it's not already there
                if statement.upper().startswith("CREATE TABLE") and "IF NOT EXISTS" not in statement.upper():
                    statement = statement.replace("CREATE TABLE", "CREATE TABLE IF NOT EXISTS", 1)

                print(f"Executing: {statement[:100]}...") # Print first 100 chars
                cursor.execute(statement)
                print("OK")
            except Error as err:
                # With IF NOT EXISTS, this error check is less critical for table existence,
                # but good for other potential DDL errors.
                print(f"Error executing statement: {err}")
                connection.rollback() # Rollback on error
                return False # Indicate failure
        connection.commit() # Commit all changes if no errors
        print("All tables created successfully (or already existed).")
        return True
    except Error as e:
        print(f"Error connecting to MySQL or during table creation: {e}")
        return False
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed.")

# --- CSV Reading and Parsing Functions ---

def parse_population_line(parts):
    """
    Parses a single line of population data (already split) and returns a dictionary
    mapping column names to their values.
    """
    # Extract fixed fields
    # parts[0] is 'BEV-VARIANTE-01' or empty
    # parts[1] is empty
    # parts[2] is 'männlich'
    # parts[3] is 'unter 1 Jahr'
    
    projection_variant = parts[0].strip() if parts[0].strip() else None
    gender = parts[2].strip()
    age_group = parts[3].strip()

    # Extract population numbers, skipping the 'e' indicators
    population_values = []
    # Data starts from index 4, and every second element is 'e'
    for i in range(4, len(parts), 2):
        try:
            # Replace comma with dot for float conversion
            val = parts[i].strip().replace(',', '.')
            population_values.append(float(val))
        except ValueError:
            population_values.append(None) # Handle cases where conversion fails

    data = {
        'ProjectionVariant': projection_variant,
        'Gender': gender,
        'AgeGroup': age_group,
    }

    # Dynamically assign population values to their respective year columns
    # SQL columns are in the format Population_YYYY_12_31
    current_year = 2022
    for value in population_values:
        col_name = f'Population_{current_year}_12_31'
        data[col_name] = value
        current_year += 1
        if current_year > 2070: # Prevent going beyond the last year defined in the SQL table
            break
    
    return data

def read_population_data_from_csv(file_path):
    """Reads population projection data from a CSV file."""
    data = []
    try:
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            # Skip the header row.
            # The original header is: "Varianten der Bevölkerungsvorausberechnung",,"Geschlecht","Altersjahre",2022 - 31.12.2022,,2023 - 31.12.2023,,...
            # The first data row is: "BEV-VARIANTE-01",,"männlich","unter 1 Jahr",385.8,e,...
            header_row = next(reader) 
            
            for row in reader:
                if row and len(row) > 3: # Ensure row is not empty and has enough columns
                    parsed_row = parse_population_line(row)
                    data.append(parsed_row)
        print(f"Read {len(data)} lines from population projection file: {file_path}")
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
    except Exception as e:
        print(f"Error reading population projection file {file_path}: {e}")
    return data

def insert_population_data(config, data_rows):
    """
    Inserts parsed population data into the PopulationProjections table.
    """
    if not data_rows:
        print("No population data to insert.")
        return

    connection = None
    try:
        connection = mysql.connector.connect(**config)
        cursor = connection.cursor()

        columns = [
            'ProjectionVariant', 'Gender', 'AgeGroup',
            'Population_2022_12_31', 'Population_2023_12_31', 'Population_2024_12_31',
            'Population_2025_12_31', 'Population_2026_12_31', 'Population_2027_12_31',
            'Population_2028_12_31', 'Population_2029_12_31', 'Population_2030_12_31',
            'Population_2031_12_31', 'Population_2032_12_31', 'Population_2033_12_31',
            'Population_2034_12_31', 'Population_2035_12_31', 'Population_2036_12_31',
            'Population_2037_12_31', 'Population_2038_12_31', 'Population_2039_12_31',
            'Population_2040_12_31', 'Population_2041_12_31', 'Population_2042_12_31',
            'Population_2043_12_31', 'Population_2044_12_31', 'Population_2045_12_31',
            'Population_2046_12_31', 'Population_2047_12_31', 'Population_2048_12_31',
            'Population_2049_12_31', 'Population_2050_12_31', 'Population_2051_12_31',
            'Population_2052_12_31', 'Population_2053_12_31', 'Population_2054_12_31',
            'Population_2055_12_31', 'Population_2056_12_31', 'Population_2057_12_31',
            'Population_2058_12_31', 'Population_2059_12_31', 'Population_2060_12_31',
            'Population_2061_12_31', 'Population_2062_12_31', 'Population_2063_12_31',
            'Population_2064_12_31', 'Population_2065_12_31', 'Population_2066_12_31',
            'Population_2067_12_31', 'Population_2068_12_31', 'Population_2069_12_31',
            'Population_2070_12_31'
        ]
        
        values_to_insert = []
        for row_data in data_rows:
            row_values = tuple(row_data.get(col, None) for col in columns)
            values_to_insert.append(row_values)

        placeholders = ', '.join(['%s'] * len(columns))
        insert_statement = f"INSERT INTO PopulationProjections ({', '.join(columns)}) VALUES ({placeholders})"

        print(f"\nInserting {len(values_to_insert)} rows into PopulationProjections...")
        cursor.executemany(insert_statement, values_to_insert)
        connection.commit()
        print(f"{cursor.rowcount} rows inserted successfully.")

    except Error as e:
        print(f"Error inserting population data: {e}")
        if connection:
            connection.rollback()
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed.")

def read_hospital_bed_details_from_csv(file_path):
    """Reads hospital bed details from a CSV file."""
    data = []
    try:
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            # Skip the header row.
            # The original header is: "Feldbezeichnung","Inhalt"
            next(reader) 
            for row in reader:
                if len(row) >= 2:
                    data.append({
                        'FieldName': row[0].strip(),
                        'Content': row[1].strip()
                    })
        print(f"Read {len(data)} lines from hospital bed details file: {file_path}")
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
    except Exception as e:
        print(f"Error reading hospital bed details file {file_path}: {e}")
    return data

def insert_hospital_bed_details_data(config, data_rows):
    """
    Inserts data into the HospitalBedDetails table.
    """
    if not data_rows:
        print("No hospital bed details to insert.")
        return

    connection = None
    try:
        connection = mysql.connector.connect(**config)
        cursor = connection.cursor()

        insert_statement = "INSERT INTO HospitalBedDetails (FieldName, Content) VALUES (%s, %s)"
        values_to_insert = [(row['FieldName'], row['Content']) for row in data_rows]

        print(f"\nInserting {len(values_to_insert)} rows into HospitalBedDetails...")
        cursor.executemany(insert_statement, values_to_insert)
        connection.commit()
        print(f"{cursor.rowcount} rows inserted successfully.")

    except Error as e:
        print(f"Error inserting hospital bed details: {e}")
        if connection:
            connection.rollback()
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed.")

def read_hospital_comprehensive_data_from_csv(file_path):
    """Reads comprehensive hospital facilities data from a CSV file."""
    data = []
    
    # Mapping CSV column names to SQL column names
    # This is for non-bed related columns
    column_mapping = {
        "Land": "Land", "Kreis": "Kreis", "Gemeinde": "Gemeinde", "Adresse_Name": "HospitalName",
        "Adresse_Name_Standort": "SiteName", "Adresse_Strasse_Standort": "SiteStreet",
        "Adresse_Haus-Nr._Standort": "SiteHouseNumber", "Adresse_Postleitzahl_Standort": "SitePostalCode",
        "Adresse_Ort_Standort": "SiteCity", "Telefonvorwahl/-nummer": "PhoneNumber",
        "E-Mail Adresse": "EmailAddress", "Internet-Adresse": "InternetAddress",
        "Traeger": "OwnerType", "T_Name": "OwnerName", "EinrichtungsTyp": "InstitutionType",
        "Allgemeine_Notfallversorgung": "GeneralEmergencyCare", "INSG": "TotalBeds"
    }

    try:
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            header = [h.strip() for h in next(reader)] # Read and clean header

            # Adjust header to handle repeated "Spezielle_Notfallversorgung" columns
            processed_header = []
            special_count = 0
            for h in header:
                if h == "Spezielle_Notfallversorgung":
                    # Use the SQL column name directly for later matching
                    processed_header.append(f"SpecialEmergencyCare_Modul{special_count + 1}")
                    special_count += 1
                else:
                    processed_header.append(h)

            for row in reader:
                if not row: continue # Skip empty rows

                row_data = {}
                for i, cell in enumerate(row):
                    if i >= len(processed_header):
                        # print(f"Warning: More columns in row than in header. Row: {row}")
                        break # Prevent IndexError

                    csv_col_name = processed_header[i]
                    
                    # Try to map directly if it's one of the fixed columns
                    sql_col_name = column_mapping.get(csv_col_name, None)

                    if sql_col_name:
                        # Convert data types for known columns
                        if sql_col_name in ["OwnerType", "InstitutionType", "GeneralEmergencyCare", "TotalBeds"]:
                            try:
                                row_data[sql_col_name] = int(cell.strip()) if cell.strip() else None
                            except ValueError:
                                row_data[sql_col_name] = None
                        else:
                            row_data[sql_col_name] = cell.strip() if cell.strip() else None
                    elif csv_col_name.startswith("SpecialEmergencyCare_Modul"):
                        row_data[csv_col_name] = cell.strip() if cell.strip() else None
                    else:
                        # Handle bed columns
                        # Remove special characters and replace dot with underscore for SQL column name
                        # E.g.: "100.0 - 118.0" -> "Beds_100_118"
                        bed_col_name_raw = csv_col_name.replace('.', '').replace('-', '_').replace(' ', '')
                        sql_bed_col_name = f"Beds_{bed_col_name_raw}"
                        try:
                            row_data[sql_bed_col_name] = int(cell.strip()) if cell.strip() else None
                        except ValueError:
                            row_data[sql_bed_col_name] = None
                data.append(row_data)
        print(f"Read {len(data)} lines from comprehensive hospital facilities file: {file_path}")
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
    except Exception as e:
        print(f"Error reading comprehensive hospital facilities file {file_path}: {e}")
    return data

def insert_hospital_comprehensive_data(config, data_rows):
    """
    Inserts data into the HospitalFacilitiesComprehensive table.
    """
    if not data_rows:
        print("No comprehensive hospital facilities data to insert.")
        return

    connection = None
    try:
        connection = mysql.connector.connect(**config)
        cursor = connection.cursor()

        # Get the list of all SQL table columns to ensure correct order
        cursor.execute("DESCRIBE HospitalFacilitiesComprehensive")
        table_columns_info = cursor.fetchall()
        sql_columns = [col[0] for col in table_columns_info if col[0] != 'id'] # Exclude 'id'

        # Prepare data for executemany
        values_to_insert = []
        for row_data in data_rows:
            row_values = []
            for col in sql_columns:
                value = row_data.get(col, None)
                if value is None and col.startswith("Beds_"):
                    # Try to find the column name in the original CSV format if SQL name not found
                    # E.g.: Beds_100_118 (SQL) -> 100.0 - 118.0 (CSV)
                    # E.g.: Beds_2953 (SQL) -> 2953 (CSV)
                    original_csv_format = col.replace("Beds_", "")
                    if '_' in original_csv_format: # If it has underscore, it might be a range or decimal
                        if original_csv_format.count('_') == 1 and original_csv_format.replace('_', '').isdigit():
                             # Case like 100_0 -> 100.0
                            original_csv_format = original_csv_format.replace('_', '.')
                        elif original_csv_format.count('_') == 2 and original_csv_format.split('_')[0].isdigit() and original_csv_format.split('_')[1].isdigit() and original_csv_format.split('_')[2].isdigit():
                            # Case like 1600_59 -> 1600.0 - 59.0
                            parts = original_csv_format.split('_')
                            original_csv_format = f"{parts[0]}.0 - {parts[1]}.0" # Assuming "X.0 - Y.0" format
                            # Correction for specific "X.0 - Y.0" formats
                            if original_csv_format == "1600.0 - 59.0":
                                original_csv_format = "1600.0 - 59.0" # Keep as is
                            elif original_csv_format == "1700.0 - 35.0":
                                original_csv_format = "1700.0 - 35.0"
                            elif original_csv_format == "1800.0 - 28.0":
                                original_csv_format = "1800.0 - 28.0"
                            elif original_csv_format == "2200.0 - 39.0":
                                original_csv_format = "2200.0 - 39.0"
                            elif original_csv_format == "2400.0 - 41.0":
                                original_csv_format = "2400.0 - 41.0"
                            elif original_csv_format == "2800.0 - 46.0":
                                original_csv_format = "2800.0 - 46.0"
                            elif original_csv_format == "1000.0 - 34.0":
                                original_csv_format = "1000.0 - 34.0"
                            elif original_csv_format == "2300.0 - 59.0": # Hypothetical example
                                original_csv_format = "2300.0 - 59.0"
                            elif original_csv_format == "8500.0 - 86.0":
                                original_csv_format = "8500.0 - 86.0"
                        else:
                            original_csv_format = original_csv_format.replace('_', '.') # Other cases with underscore
                    
                    value = row_data.get(original_csv_format, None)
                row_values.append(value)
            values_to_insert.append(tuple(row_values))

        placeholders = ', '.join(['%s'] * len(sql_columns))
        insert_statement = f"INSERT INTO HospitalFacilitiesComprehensive ({', '.join(sql_columns)}) VALUES ({placeholders})"

        print(f"\nInserting {len(values_to_insert)} rows into HospitalFacilitiesComprehensive...")
        cursor.executemany(insert_statement, values_to_insert)
        connection.commit()
        print(f"{cursor.rowcount} rows inserted successfully.")

    except Error as e:
        print(f"Error inserting comprehensive hospital facilities data: {e}")
        if connection:
            connection.rollback()
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed.")

def read_hospital_summarized_data_from_csv(file_path):
    """Reads summarized hospital facilities data from a CSV file."""
    data = []
    
    # Mapping CSV column names to SQL column names
    column_mapping = {
        "Land": "Land", "Kreis": "Kreis", "Gemeinde": "Gemeinde", "Adresse_Name": "HospitalName",
        "Adresse_Straße": "Street", "Adresse_Haus-Nr.": "HouseNumber",
        "Adresse_Postleitzahl": "PostalCode", "Adresse_Ort": "City",
        "Telefonvorwahl/-nummer": "PhoneNumber", "E-Mail Adresse": "EmailAddress",
        "Internet-Adresse": "InternetAddress", "Traeger": "OwnerType", "T-Name": "OwnerName",
        "EinrichtungsTyp": "InstitutionType", "INSG": "TotalBeds"
    }

    try:
        with open(file_path, mode='r', encoding='utf-8') as file:
            reader = csv.reader(file)
            header = [h.strip() for h in next(reader)] # Read and clean header

            for row in reader:
                if not row: continue # Skip empty rows

                row_data = {}
                for i, cell in enumerate(row):
                    if i >= len(header):
                        # print(f"Warning: More columns in row than in header. Row: {row}")
                        break # Prevent IndexError

                    csv_col_name = header[i]
                    sql_col_name = column_mapping.get(csv_col_name, None)

                    if sql_col_name:
                        # Convert data types for known columns
                        if sql_col_name in ["OwnerType", "InstitutionType", "TotalBeds"]:
                            try:
                                row_data[sql_col_name] = int(cell.strip()) if cell.strip() else None
                            except ValueError:
                                row_data[sql_col_name] = None
                        else:
                            row_data[sql_col_name] = cell.strip() if cell.strip() else None
                    else:
                        # Handle bed columns
                        # Remove special characters and replace dot with underscore for SQL column name
                        bed_col_name_raw = csv_col_name.replace('.', '').replace('-', '_').replace(' ', '')
                        sql_bed_col_name = f"Beds_{bed_col_name_raw}"
                        try:
                            row_data[sql_bed_col_name] = int(cell.strip()) if cell.strip() else None
                        except ValueError:
                            row_data[sql_bed_col_name] = None
                data.append(row_data)
        print(f"Read {len(data)} lines from summarized hospital facilities file: {file_path}")
    except FileNotFoundError:
        print(f"Error: File not found - {file_path}")
    except Exception as e:
        print(f"Error reading summarized hospital facilities file {file_path}: {e}")
    return data

def insert_hospital_summarized_data(config, data_rows):
    """
    Inserts data into the HospitalFacilitiesSummarized table.
    """
    if not data_rows:
        print("No summarized hospital facilities data to insert.")
        return

    connection = None
    try:
        connection = mysql.connector.connect(**config)
        cursor = connection.cursor()

        # Get the list of all SQL table columns to ensure correct order
        cursor.execute("DESCRIBE HospitalFacilitiesSummarized")
        table_columns_info = cursor.fetchall()
        sql_columns = [col[0] for col in table_columns_info if col[0] != 'id'] # Exclude 'id'

        # Prepare data for executemany
        values_to_insert = []
        for row_data in data_rows:
            row_values = []
            for col in sql_columns:
                value = row_data.get(col, None)
                if value is None and col.startswith("Beds_"):
                    # Try to find the column name in the original CSV format if SQL name not found
                    original_csv_format = col.replace("Beds_", "")
                    if '_' in original_csv_format: # If it has underscore, it might be a range or decimal
                        if original_csv_format.count('_') == 1 and original_csv_format.replace('_', '').isdigit():
                            original_csv_format = original_csv_format.replace('_', '.')
                        elif original_csv_format.count('_') == 2 and original_csv_format.split('_')[0].isdigit() and original_csv_format.split('_')[1].isdigit() and original_csv_format.split('_')[2].isdigit():
                            parts = original_csv_format.split('_')
                            original_csv_format = f"{parts[0]}.0 - {parts[1]}.0" # Assuming "X.0 - Y.0" format
                            # Handle the specific "8500.0 - 86.0" case
                            if original_csv_format == "8500.0 - 86.0":
                                original_csv_format = "8500.0 - 86.0"
                        else:
                            original_csv_format = original_csv_format.replace('_', '.') # Other cases with underscore
                    
                    value = row_data.get(original_csv_format, None)
                row_values.append(value)
            values_to_insert.append(tuple(row_values))

        placeholders = ', '.join(['%s'] * len(sql_columns))
        insert_statement = f"INSERT INTO HospitalFacilitiesSummarized ({', '.join(sql_columns)}) VALUES ({placeholders})"

        print(f"\nInserting {len(values_to_insert)} rows into HospitalFacilitiesSummarized...")
        cursor.executemany(insert_statement, values_to_insert)
        connection.commit()
        print(f"{cursor.rowcount} rows inserted successfully.")

    except Error as e:
        print(f"Error inserting summarized hospital facilities data: {e}")
        if connection:
            connection.rollback()
    finally:
        if connection and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection closed.")


# --- Main Execution Block ---
if __name__ == "__main__":
    # Step 1: Ensure the database exists
    if create_database_if_not_exists(DB_CONFIG):
        # Step 2: Create the tables within that database
        if create_tables(DB_CONFIG, SQL_CREATE_TABLES):
            print("\nDatabase and tables setup complete.")
            
            # Step 3: Read and insert data from each CSV file
            
            # Population Projection Data (File 1)
            for file_path in CSV_FILE_PATHS['population']:
                population_data = read_population_data_from_csv(file_path)
                insert_population_data(DB_CONFIG, population_data)

            # Hospital Bed Details (File 2)
            for file_path in CSV_FILE_PATHS['bed_details']:
                bed_details_data = read_hospital_bed_details_from_csv(file_path)
                insert_hospital_bed_details_data(DB_CONFIG, bed_details_data)

            # Comprehensive Hospital Facilities Data (File 3)
            for file_path in CSV_FILE_PATHS['comprehensive']:
                comprehensive_data = read_hospital_comprehensive_data_from_csv(file_path)
                insert_hospital_comprehensive_data(DB_CONFIG, comprehensive_data)

            # Summarized Hospital Facilities Data (File 4)
            for file_path in CSV_FILE_PATHS['summarized']:
                summarized_data = read_hospital_summarized_data_from_csv(file_path)
                insert_hospital_summarized_data(DB_CONFIG, summarized_data)

            print("\nCSV data import process completed.")
        else:
            print("\nTable creation failed.")
    else:
        print("\nDatabase creation failed, aborting table setup.")



Attempting to create database 'hospital_data' if it doesn't exist...
Database 'hospital_data' ensured to exist.
Executing: -- Table for File 1: Population Projection Data
-- This table stores population counts based on diff...
OK
Executing: -- Table for File 2: Hospital Bed Details
-- This table serves as a lookup for detailed hospital dep...
OK
Executing: -- Table for File 3: Comprehensive Hospital Facilities Data
-- This table stores comprehensive infor...
OK
Executing: -- Table for File 4: Summarized Hospital Facilities Data
-- This table provides a summarized view of...
OK
All tables created successfully (or already existed).
MySQL connection closed.

Database and tables setup complete.
Read 8899 lines from population projection file: C:\Users\user\Desktop\A_AI_DDSP\data\processed\Processed\cleaned_Bevölkerung forecasts_ALL VAR.csv

Inserting 8899 rows into PopulationProjections...
8899 rows inserted successfully.
MySQL connection closed.
Read 330 lines from population projection f

In [2]:
import pyodbc

# Replace with the path to your .accdb file
db_path = r"C:\Users\user\Desktop\A_AI_DDSP\Dataset\raw\DRG-Statistik (Patientenvolumen)\2021\Versand_19_DRG_PatKreis_2021.accdb"

# import pyodbc
import csv

# Path to your Access database file
# db_path = r"C:\path\to\Versand_19_DRG_PatKreis_2021.accdb"

# Connect to the database
conn = pyodbc.connect(
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=' + db_path
)
cursor = conn.cursor()

# Get the list of tables
cursor.execute("SELECT name FROM MSysObjects WHERE type=1 AND name NOT LIKE 'MSys%'")
tables = [row[0] for row in cursor.fetchall()]

# Loop through tables and extract data
for table in tables:
    print(f"Extracting data from table: {table}")
    
    # Fetch all data
    cursor.execute(f"SELECT * FROM {table}")
    rows = cursor.fetchall()

    # Save data to CSV
    with open(f"{table}.csv", "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([column[0] for column in cursor.description])  # Write headers
        writer.writerows(rows)  # Write data

    print(f"Data from {table} exported successfully to {table}.csv")

# Close the connection
conn.close()
print("Database connection closed.")


InterfaceError: ('IM002', '[IM002] [Microsoft][ODBC Driver Manager] Data source name not found and no default driver specified (0) (SQLDriverConnect)')

In [3]:
import pyodbc
print(pyodbc.drivers())

['SQL Server', 'ODBC Driver 18 for SQL Server']
